In [1]:
#import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gmaps

from config import census_key,g_key

gmaps.configure(g_key)

In [2]:
data = pd.read_csv("../Output/census_data.csv")
byzipcode = data.copy()
races = byzipcode.columns[4:]
for race in races:
    byzipcode[f'{race}%'] = byzipcode[race]/byzipcode['population']

lat_log = pd.read_csv('../Resources/us-zip-code-latitude-and-longitude.csv', sep = ';')
lat_log = lat_log[['Zip','geopoint']]

byzipcode = pd.merge(lat_log,byzipcode, how = 'right', right_on = 'zipcode', left_on = 'Zip')
byzipcode.drop(columns= 'zipcode', inplace = True)
byzipcode.rename(columns = {'Zip':'zipcode'},inplace = True)
byzipcode.head()

,zipcode,geopoint,city,population,per_capita_income,white,african_american,native_american,asian,native_hawaiian_pacific_islander,...,hispanic_or_latino,white%,african_american%,native_american%,asian%,native_hawaiian_pacific_islander%,other_race%,two_or_more_races%,white_not_hispanic%,hispanic_or_latino%
0,46214,"39.792993,-86.28575",Indianapolis,24202.0,26462.0,12760.0,8746.0,30.0,446.0,189.0,...,3312.0,0.527229,0.361375,0.001240,0.018428,0.007809,0.035865,0.048054,0.450459,0.136848
1,75087,"32.93382,-96.45446",Dallas,33638.0,39436.0,30220.0,954.0,90.0,873.0,0.0,...,4076.0,0.898389,0.028361,0.002676,0.025953,0.000000,0.017926,0.026696,0.797342,0.121172
2,30336,"33.744007,-84.56032",Atlanta,234.0,42524.0,33.0,186.0,0.0,7.0,0.0,...,11.0,0.141026,0.794872,0.000000,0.029915,0.000000,0.000000,0.034188,0.123932,0.047009
3,19104,"39.961612,-75.19957",Philadelphia,54311.0,15478.0,20419.0,22962.0,291.0,7662.0,62.0,...,2766.0,0.375964,0.422787,0.005358,0.141076,0.001142,0.015430,0.038243,0.345823,0.050929
4,19146,"39.938512,-75.18067",Philadelphia,38873.0,45507.0,18549.0,16645.0,120.0,1701.0,35.0,...,1799.0,0.477169,0.428189,0.003087,0.043758,0.000900,0.015023,0.031873,0.446042,0.046279


In [3]:
seattle_df = pd.read_csv("../Output/seattle_restaurants.csv")
phila_df = pd.read_csv("../Output/philadelphia_restaurants.csv")
dallas_df = pd.read_csv("../Output/dallas_restaurants.csv")
indie_df = pd.read_csv("../Output/indianapolis_restaurants.csv")
alt_df = pd.read_csv("../Output/atlanta_restaurants.csv")

yelp_df = pd.concat([seattle_df,phila_df,dallas_df,indie_df,alt_df])
yelp_df = yelp_df.reset_index()
yelp_df.drop(columns = ['index','Unnamed: 0'], inplace = True)
yelp_df

,name,city,latitude,longitude,zipcode,ethnicity,price,rating
0,A + Hong Kong Kitchen,Seattle,47.598540,-122.326520,98104,chinese,$,4.0
1,Harbor City Restaurant,Seattle,47.598210,-122.323390,98104,chinese,$$,4.0
2,Dough Zone,Seattle,47.613170,-122.332150,98101,chinese,$$,4.0
3,Dough Zone,Seattle,47.597880,-122.327490,98104,chinese,$$,4.0
4,Biang Biang Noodles,Seattle,47.613937,-122.324239,98122,chinese,NaN,4.0
...,...,...,...,...,...,...,...,...
4825,T1 Bento,Atlanta,33.907766,-84.289732,30341,thai,$,4.0
4826,Saigon Cafe - Cumberland,Atlanta,33.887040,-84.475595,30080,thai,$$,3.5
4827,Thai Cafe,Atlanta,33.879660,-84.467664,30339,thai,NaN,2.5
4828,Tin Drum Asian Kitchen - Akers Mill Square,Atlanta,33.882804,-84.462033,30339,thai,$,2.5


In [4]:
df = pd.merge(yelp_df,byzipcode, how = 'left',on = 'zipcode')
df

,name,city_x,latitude,longitude,zipcode,ethnicity,price,rating,geopoint,city_y,...,hispanic_or_latino,white%,african_american%,native_american%,asian%,native_hawaiian_pacific_islander%,other_race%,two_or_more_races%,white_not_hispanic%,hispanic_or_latino%
0,A + Hong Kong Kitchen,Seattle,47.598540,-122.326520,98104,chinese,$,4.0,"47.60252,-122.32855",Seattle,...,1137.0,0.525099,0.103224,0.017958,0.250354,0.003323,0.029765,0.070277,0.486143,0.080387
1,Harbor City Restaurant,Seattle,47.598210,-122.323390,98104,chinese,$$,4.0,"47.60252,-122.32855",Seattle,...,1137.0,0.525099,0.103224,0.017958,0.250354,0.003323,0.029765,0.070277,0.486143,0.080387
2,Dough Zone,Seattle,47.613170,-122.332150,98101,chinese,$$,4.0,"47.61067,-122.33438",Seattle,...,609.0,0.680894,0.060507,0.012273,0.180972,0.003049,0.014462,0.047842,0.653924,0.047608
3,Dough Zone,Seattle,47.597880,-122.327490,98104,chinese,$$,4.0,"47.60252,-122.32855",Seattle,...,1137.0,0.525099,0.103224,0.017958,0.250354,0.003323,0.029765,0.070277,0.486143,0.080387
4,Biang Biang Noodles,Seattle,47.613937,-122.324239,98122,chinese,NaN,4.0,"47.61157,-122.30406",Seattle,...,2863.0,0.650751,0.122890,0.011212,0.101942,0.005800,0.026489,0.080917,0.610979,0.074133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,T1 Bento,Atlanta,33.907766,-84.289732,30341,thai,$,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4826,Saigon Cafe - Cumberland,Atlanta,33.887040,-84.475595,30080,thai,$$,3.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4827,Thai Cafe,Atlanta,33.879660,-84.467664,30339,thai,NaN,2.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4828,Tin Drum Asian Kitchen - Akers Mill Square,Atlanta,33.882804,-84.462033,30339,thai,$,2.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#zipcodes where there is no census data or is not actually part of the city
df[df['city_y'].isnull()]['zipcode'].unique()


array([98004, 98108, 98040, 98033, 98034, 98056, 98127, 98155, 98006,
       98005, 98195, 19004,  8103, 19003, 19050,  8078,  8110,  8035,
        8108,  8033, 19018, 19032,  8106, 19023, 19082,  8109, 19036,
       19096,  8093, 19076,  8002, 19083,  8007,  8107,  8031, 19026,
        8105,  8104,  8030, 19072, 19041, 19176, 75244, 75042, 75211,
       75217, 75254, 75270, 46815, 75044, 75001, 75390, 75149, 46203,
       46250, 46227, 46240, 46077, 47906, 47305, 47901, 46032, 47904,
       46107, 46228, 73505, 46216, 46206, 46255, 45245, 46262, 30329,
       30341, 30033, 30340, 30360, 30339, 30345, 30328, 30080, 30030,
       30302, 30358, 30366, 30067, 30084, 30079, 30060, 31634, 32011,
       30322, 30036, 30144, 30361, 30101, 30034])

In [6]:
#drop these rows. can't use simple drop na as we want to keep NaA in price
df.drop(index = df[df['city_y'].isnull()].index, inplace = True)
df.drop(columns = ['city_y'], inplace = True)
df.rename(columns = {'name' : 'restaurant_name',
                     'city_x' : 'city', 
                     'geopoint': 'zipcode geopoint', 
                     'ethnicity':'restaurant_ethnicity'}, inplace = True)

In [7]:
df.columns

Index(['restaurant_name', 'city', 'latitude', 'longitude', 'zipcode',
       'restaurant_ethnicity', 'price', 'rating', 'zipcode geopoint',
       'population', 'per_capita_income', 'white', 'african_american',
       'native_american', 'asian', 'native_hawaiian_pacific_islander',
       'other_race', 'two_or_more_races', 'white_not_hispanic',
       'hispanic_or_latino', 'white%', 'african_american%', 'native_american%',
       'asian%', 'native_hawaiian_pacific_islander%', 'other_race%',
       'two_or_more_races%', 'white_not_hispanic%', 'hispanic_or_latino%'],
      dtype='object')

In [8]:
#want to reorder columns
df = df[['restaurant_name','restaurant_ethnicity','latitude', 'longitude',
        'zipcode','zipcode geopoint','city','price', 'rating', 
       'population', 'per_capita_income', 'white', 'african_american',
       'native_american', 'asian', 'native_hawaiian_pacific_islander',
       'other_race', 'two_or_more_races', 'white_not_hispanic',
       'hispanic_or_latino', 'white%', 'african_american%', 'native_american%',
       'asian%', 'native_hawaiian_pacific_islander%', 'other_race%',
       'two_or_more_races%', 'white_not_hispanic%', 'hispanic_or_latino%']]